In [1]:
%pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install tabulate scikit-learn matplotlib pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import moco

In [4]:
# from experiment_fraud import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
import pandas as pd
import numpy as np
from moco.early_exit_model import EarlyExitModel
import pickle as pkl
import time
from sklearn.metrics import precision_recall_curve, precision_recall_fscore_support

In [5]:
def load_dataset():
    df = pd.read_csv('/Users/samrandall/Desktop/creditcard.csv')
    X = df[[col for col in df.columns if col.startswith('V')]].to_numpy()
    y = df['Class'].to_numpy()
    return X, y

In [6]:
X, y = load_dataset()
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.12)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.15)
train_x.shape, val_x.shape, test_x.shape

((213035, 28), (37595, 28), (34177, 28))

In [7]:
def find_p_r_combo_where_r_greater_than(x, y, model, recall_threshold: float):
    probs = model.predict_proba(x)[:, 1]
    precisions, recalls, thresholds = precision_recall_curve(y, probs)

    precisions = precisions[:-1]
    recalls = recalls[:-1]

    mask = recalls >= recall_threshold

    if np.any(mask):
        # Among the qualifying points, pick the one with highest precision
        best_idx = np.argmax(precisions[mask])
        threshold = thresholds[mask][best_idx]
        return threshold
    else:
        # No threshold can reach desired recall
        return None


In [8]:
architectures = [(16,), (32,), (64,), (128,), (256,), (512,), (64, 32), (128, 64), (256, 128)]
models = {}
for arch in architectures:
    mlp = MLPClassifier(arch)
    mlp.fit(train_x, train_y)
    models[arch] = mlp

In [9]:
rows = []
for arch, mlp in models.items():
    t = find_p_r_combo_where_r_greater_than(val_x, val_y, mlp, 0.75)
    test_pred = mlp.predict_proba(test_x)[:, 1] > t
    p, r, f, s = precision_recall_fscore_support(test_y, test_pred)

    rows.append({
        "Architecture": arch,
        "Precision": p[1],
        "Recall": r[1],
        "Fscore": f[1],
        "Support": s[1]
    })


    val_pred = mlp.predict_proba(val_x)[:, 1]
    p, r, t = precision_recall_curve(val_y, val_pred)

pr_results = pd.DataFrame(rows)
pr_results.to_csv("pr_results.csv")
pr_results[["Architecture", "Precision", "Recall"]].to_markdown('precision_recall.md')


In [10]:
rows = []
for arch, mlp in models.items():

    eem = EarlyExitModel(mlp)
    eem.add_linear_prediction_rule(train_x, (0,))

    # Warmup.
    for i in range(5):
        b_predictions = mlp.predict(X)
        eem.predict(X)

    # Baseline
    start = time.time()
    baseline_times = []
    for i in range(5):
        s = time.time()
        b_predictions = mlp.predict(X)
        e = time.time()
        baseline_times.append(e - s)

    end = time.time()
    baseline_time = (end - start) / 5

    for i in range(5):
        b_predictions = mlp.predict(X)
        eem.predict(X)

    # Experiment
    start = time.time()
    total_time = 0
    experiment_times = []
    for i in range(5):
        s = time.time()
        e_predictions = eem.predict(X)
        e = time.time()
        total_time += e - start
        experiment_times.append(e - s)
    
    end = time.time()

    experimental_time = (end - start) / 5
    agreement = (b_predictions == e_predictions).mean()
    rows.append({"MLP Architecture": arch,
        "sklearn MLP time (s)": np.round(baseline_time, 6),
        "Gated MLP time (s)": np.round(experimental_time, 6),
        "adherence": agreement,
        "sklearn std": np.std(baseline_times),
        "gated std": np.std(experiment_times)
    })

results = pd.DataFrame(rows)


results.to_csv('out.csv')


In [11]:
for_report = results[["MLP Architecture", "sklearn MLP time (s)", "Gated MLP time (s)", 'adherence']]
for_report['Speedup'] = for_report['sklearn MLP time (s)'] / for_report["Gated MLP time (s)"]
for_report= for_report[["MLP Architecture", "sklearn MLP time (s)", "Gated MLP time (s)", "Speedup", 'adherence']]
for_report.to_markdown('output.md')

/var/folders/gk/4qpy_m5j67d9h1h0g9j07wzr0000gn/T/ipykernel_27208/4236618802.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  for_report['Speedup'] = for_report['sklearn MLP time (s)'] / for_report["Gated MLP time (s)"]


In [12]:
for_report

,MLP Architecture,sklearn MLP time (s),Gated MLP time (s),Speedup,adherence
0,"(16,)",0.020058,0.040558,0.494551,1.000000
1,"(32,)",0.026208,0.039290,0.667040,0.999996
2,"(64,)",0.053525,0.054338,0.985038,1.000000
3,"(128,)",0.090559,0.052611,1.721294,0.999996
4,"(256,)",0.176512,0.133290,1.324270,1.000000
5,"(512,)",0.438908,0.367009,1.195905,1.000000
6,"(64, 32)",0.067092,0.068520,0.979159,1.000000
7,"(128, 64)",0.149170,0.131838,1.131464,0.999996
8,"(256, 128)",0.308446,0.273897,1.126139,0.999996
